# Recommender System.  Code Along Project.

### (Henri's Version)



In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("rec").getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
# ALS stands for Alternating Least Squares matrix factorization.

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
# Evaluator for Regression, which expects two input columns:  prediction and label.

In [5]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.describe().show()
# We have 1501 samples.
# This is actually considered to be very small for a Recommender System.

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)

In [9]:
# Create our alternating least squares model
als = ALS(maxIter=5, regParam=0.01,
          userCol="userId", itemCol="movieId", ratingCol="rating")

In [10]:
als_fitted = als.fit(dataset=train_data)

In [11]:
# Transforms the input dataset with optional parameters
predictions = als_fitted.transform(dataset=test_data)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   3.0|     8|  3.0772996|
|     85|   1.0|    28|  5.4000583|
|     85|   3.0|     1|  1.8228228|
|     85|   1.0|    13|  1.7642412|
|     85|   3.0|     6|  2.6746016|
|     85|   3.0|    21|  3.1963556|
|     65|   2.0|     5|  1.6886216|
|     53|   1.0|     6|  0.2523802|
|     53|   5.0|     8|  0.4430682|
|     53|   3.0|    14|   4.293657|
|     78|   1.0|    28|-0.19155872|
|     78|   1.0|    13|  0.7180194|
|     34|   1.0|    14|  3.0078955|
|     34|   1.0|     0|    2.39131|
|     81|   3.0|    26|-0.12628022|
|     81|   2.0|    29|  0.7365723|
|     81|   4.0|    11|-0.34560478|
|     28|   1.0|     2|  2.7639906|
|     28|   1.0|     0|  1.5114405|
|     28|   5.0|    18|  0.8054819|
+-------+------+------+-----------+
only showing top 20 rows



### Formally evaluate this model

In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [14]:
rmse = evaluator.evaluate(dataset=predictions)

In [15]:
print("RMSE = {:.3f}.".format(rmse))

# RMSE = 1.723
# Rather bad for a recommendation system.
# But keep in mind that our dataset is very small.

RMSE = 1.839.


### How do we use our model and results on "fresh" new users?

In [16]:
single_user = test_data.filter(test_data["userId"] == 11).select(["movieId", "userId"])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     72|    11|
|     79|    11|
|     81|    11|
|     86|    11|
|     97|    11|
+-------+------+



In [17]:
# Now predict whether or not userId 11 will enjoy the movies with movieId:
# [72, 79, 81, 86, 97]
# What do you think userId 11 star ratings would be for [72, 79, 81, 86, 97]?

In [18]:
star_ratings = als_fitted.transform(single_user)

In [19]:
star_ratings.orderBy("prediction", ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     79|    11|   2.170864|
|     86|    11|  1.6567501|
|     97|    11|   0.274612|
|     72|    11| 0.20933473|
|     81|    11|-0.34560478|
+-------+------+-----------+



**Coldstart problem:**

What do you do with users that are completely new to your platform, and have not viewed any movies yet on your platform?

This is a domain-knowledge problem, rather than an algorithmic problem.